### Connect Database

In [ ]:
import pyodbc
server = 'group15-server.database.windows.net'  
database = 'group15_Yelp_Database'  
username = 'group15'  
password = 'Badam15123'  
driver = '{ODBC Driver 18 for SQL Server}' 
conn = pyodbc.connect(
    f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};'
    'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=60;'
)
cursor = conn.cursor()
print("Connected.")

Connected.


In [4]:
# Run Query Function
import pandas as pd
def run_query(conn, sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    return pd.DataFrame.from_records(rows, columns=columns)

# Usage:
#df = run_query(conn, query)

## Milestone 3

1. Target City: Buffalo

Buffalo is strategically located near the Canadian border, making it a hub for cross-border travelers and international traffic. Its position as a major passage encourages frequent visitation and high footfall, especially from tourists and commuters traveling to and from Canada.

The city also boasts a significant Polish population, creating a built-in customer base that appreciates authentic Polish cuisine and cultural connections.

Buffalo's culinary scene is growing but shows gaps in ethnic diversity, suggesting an opportunity for differentiation by addressing underserved tastes and community representation.


2. Restaurant Concept

Cuisine: Polish-American fusion—offering classic Polish dishes infused with American flavors or formats. Examples could include pierogi with locally-inspired fillings, kielbasa sandwiches, Polish-American burgers, or fusion desserts.

Price Point: Individual meals under $20, maintaining accessibility and affordability for a variety of customers (students, families, travelers).

Style: Fast-Casual Hybrid: Emphasize speed and price efficiency for tourists in transit, but offer a cozy area for locals to relax and connect.

3. Target Customers

Dual focus on Buffalo locals—especially the Polish community—ensures steady, repeat business and cultural authenticity.

Tourists and commuters passing through Buffalo represent transient demand, bringing diversity and exposure to new customers. Strategic location near major highways and border crossings increases visibility and convenience for this segment.

Both groups are likely to appreciate accessible pricing, authentic cuisine, and efficient service, turning a restaurant visit into a memorable cultural experience.

In [ ]:
# 1. Count of Polish Restaurants in Buffalo
polish_restaurant_count_query = """
SELECT COUNT(*) AS polish_restaurant_count
FROM business
WHERE city = 'Buffalo'
  AND categories LIKE '%Polish%';
"""
polish_restaurant_count_df = run_query(conn, polish_restaurant_count_query)
polish_restaurant_count_df.head()


Context: Reveals saturation. If the count is low, there’s a strong niche opportunity.

In [ ]:
# 2. Average Rating and Reviews for Polish Restaurants
polish_avg_rating_reviews_query = """
SELECT AVG(stars) AS avg_rating, AVG(review_count) AS avg_reviews
FROM business
WHERE city = 'Buffalo'
  AND categories LIKE '%Polish%';
"""
polish_avg_rating_reviews_df = run_query(conn, polish_avg_rating_reviews_query)
polish_avg_rating_reviews_df.head()


Context: Benchmarks customer satisfaction and engagement, helping set goals for quality and visibility.

In [ ]:
# 3. Distribution of Polish Restaurant Ratings
polish_rating_distribution_query = """
SELECT stars, COUNT(*) AS restaurant_count
FROM business
WHERE city = 'Buffalo'
  AND categories LIKE '%Polish%'
GROUP BY stars
ORDER BY stars DESC;
"""
polish_rating_distribution_df = run_query(conn, polish_rating_distribution_query)
polish_rating_distribution_df.head()


Context: Reveals trends in quality—are most Polish restaurants highly rated or do they struggle?

In [ ]:
# 4. Neighborhoods with No Polish Restaurants
no_polish_neighborhoods_query = """
SELECT DISTINCT b.address
FROM business b
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Restaurant%'
  AND b.address NOT IN (
      SELECT address
      FROM business
      WHERE city = 'Buffalo' AND categories LIKE '%Polish%'
  );
"""
no_polish_neighborhoods_df = run_query(conn, no_polish_neighborhoods_query)
no_polish_neighborhoods_df.head()


Context: Identifies “restaurant deserts” suitable for a new Polish restaurant.

In [ ]:
# 5. Most Valued Attributes Among Top-Rated Polish Restaurants
polish_top_attributes_query = """
SELECT a.attribute_key, COUNT(*) AS freq
FROM business b
JOIN attributes a ON b.business_id = a.business_id
WHERE b.city = 'Buffalo'
  AND b.stars >= 4
  AND b.categories LIKE '%Polish%'
GROUP BY a.attribute_key
ORDER BY freq DESC;
"""
polish_top_attributes_df = run_query(conn, polish_top_attributes_query)
polish_top_attributes_df.head()


Context: Prioritizes operational features (like “Takes Reservations”) based on what works.



In [ ]:
# 6. Common Complaints in 1–2 Star Reviews (Service/Price/Wait/Portion)
polish_common_complaints_query = """
SELECT COUNT(*) AS complaint_count
FROM review r
JOIN business b ON r.business_id = b.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%'
  AND r.stars <= 2
  AND (r.text LIKE '%wait%' OR r.text LIKE '%price%' OR r.text LIKE '%service%' OR r.text LIKE '%portion%');
"""
polish_common_complaints_df = run_query(conn, polish_common_complaints_query)
polish_common_complaints_df.head()


In [ ]:
Context: Reveals recurring service or food issues to avoid.

In [ ]:
# 7. Top Keywords in Positive (4+ Star) Reviews for Polish Restaurants
polish_positive_keywords_query = """
SELECT TOP 10 r.text, COUNT(*) AS freq
FROM review r
JOIN business b ON r.business_id = b.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%'
  AND r.stars >= 4
GROUP BY r.text
ORDER BY freq DESC;
"""
polish_positive_keywords_df = run_query(conn, polish_positive_keywords_query)
polish_positive_keywords_df.head()


Context: Determines which aspects (food, staff, atmosphere) customers appreciate most.



In [ ]:
# 8. Polish Restaurant Sub-Categories with Few Competitors, High Ratings
polish_underserved_subcategory_query = """
SELECT categories, COUNT(*) AS restaurant_count, AVG(stars) AS avg_rating
FROM business
WHERE city = 'Buffalo'
  AND categories LIKE '%Polish%'
GROUP BY categories
HAVING COUNT(*) < 3 AND AVG(stars) > 4
ORDER BY avg_rating DESC;
"""
polish_underserved_subcategory_df = run_query(conn, polish_underserved_subcategory_query)
polish_underserved_subcategory_df.head()


Context: Finds possible segment “gaps” for specialization.

In [ ]:
# 9. Peak Check-in Times for Polish Restaurants
polish_peak_checkins_query = """
SELECT h.day, h.open_time, h.close_time, COUNT(c.business_id) AS checkin_count
FROM business b
JOIN hours h ON b.business_id = h.business_id
JOIN checkin c ON b.business_id = c.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%'
GROUP BY h.day, h.open_time, h.close_time
ORDER BY checkin_count DESC;
"""
polish_peak_checkins_df = run_query(conn, polish_peak_checkins_query)
polish_peak_checkins_df.head()


Context: Guides staffing, service style, and promotional timing.

In [ ]:
# 10. Longevity and Rating for Polish Restaurants
polish_longevity_rating_query = """
SELECT b.name, MIN(r.date) AS open_since, AVG(r.stars) AS avg_rating
FROM business b
JOIN review r ON b.business_id = r.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%'
GROUP BY b.name
ORDER BY open_since ASC;
"""
polish_longevity_rating_df = run_query(conn, polish_longevity_rating_query)
polish_longevity_rating_df.head()


Context: Understands longevity and sets expectations for performance as the business matures.

In [ ]:
# 11. Busiest Days for Polish Restaurants (by Check-ins)
polish_busiest_days_query = """
SELECT h.day, COUNT(c.business_id) AS total_checkins
FROM business b
JOIN hours h ON b.business_id = h.business_id
JOIN checkin c ON b.business_id = c.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%'
GROUP BY h.day
ORDER BY total_checkins DESC;
"""
polish_busiest_days_df = run_query(conn, polish_busiest_days_query)
polish_busiest_days_df.head()


Context: Pinpoints which days see the highest customer volume, guiding weekly specials or staffing plans.

In [ ]:
# 12. Average Review Length and Average Rating for Polish Restaurants
polish_review_length_rating_query = """
SELECT AVG(LEN(r.text)) AS avg_review_length, AVG(r.stars) AS avg_rating
FROM review r
JOIN business b ON r.business_id = b.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%';
"""
polish_review_length_rating_df = run_query(conn, polish_review_length_rating_query)
polish_review_length_rating_df.head()


Context: Longer reviews may indicate deeper engagement (positive or negative); helps interpret feedback quality.



In [ ]:
# 13. Currently Open vs Closed Polish Restaurants
polish_open_closed_query = """
SELECT is_open, COUNT(*) AS restaurant_count
FROM business
WHERE city = 'Buffalo'
  AND categories LIKE '%Polish%'
GROUP BY is_open;
"""
polish_open_closed_df = run_query(conn, polish_open_closed_query)
polish_open_closed_df.head()


Context: Shows current market activity and potential risk (e.g., closures may indicate obstacles in the niche).

In [ ]:
# 14. Attributes Linked to Closed Polish Restaurants
polish_closed_attributes_query = """
SELECT a.attribute_key, COUNT(*) AS freq
FROM business b
JOIN attributes a ON b.business_id = a.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%'
  AND b.is_open = 0
GROUP BY a.attribute_key
ORDER BY freq DESC;
"""
polish_closed_attributes_df = run_query(conn, polish_closed_attributes_query)
polish_closed_attributes_df.head()


Context: Reveals whether specific features (e.g., lack of kid-friendliness, poor accessibility) are linked to unsuccessful restaurants.

In [ ]:
# 15. Average Days Since Last Check-in for Polish Restaurants
polish_days_since_checkin_query = """
SELECT AVG(DATEDIFF(day, MAX(CAST(c.date AS DATETIME)), GETDATE())) AS avg_days_since_last_checkin
FROM business b
JOIN checkin c ON b.business_id = c.business_id
WHERE b.city = 'Buffalo'
  AND b.categories LIKE '%Polish%';
"""
polish_days_since_checkin_df = run_query(conn, polish_days_since_checkin_query)
polish_days_since_checkin_df.head()


Context: Measures current customer engagement—recent check-ins indicate active demand, while long intervals signal decline.